In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

In [2]:
include("load_network.jl")

load_ta_network (generic function with 2 methods)

In [3]:
using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

create_graph (generic function with 1 method)

In [4]:
function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

get_vector (generic function with 1 method)

In [5]:
ta_data = load_ta_network("Sioux Falls simplified")

TA_Data("Sioux Falls simplified",12,12,1,30,[1,1,2,2,3,3,3,4,4,4  …  9,9,9,10,10,11,11,11,12,12],[2,3,1,6,1,4,12,3,5,11  …  5,8,10,9,11,4,10,12,3,11],[25900.2,23403.5,25900.2,4958.18,23403.5,17110.5,23403.5,17110.5,17782.8,4908.83  …  10000.0,5050.19,13915.8,13915.8,10000.0,4908.83,10000.0,4908.83,23403.5,4908.83],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  5.0,10.0,3.0,3.0,5.0,6.0,5.0,6.0,4.0,6.0],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  5.0,10.0,3.0,3.0,5.0,6.0,5.0,6.0,4.0,6.0],[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15  …  0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15],[4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0  …  4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[1,1,1,1,1,1,1,1,1,1  …  1,1,1,1,1,1,1,1,1,1],360600.0,12x12 Array{Float64,2}:
    0.0  100.0  100.0   500.0   200.0  …   500.0  1300.0   500.0   

In [6]:
# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

4.23133528710744e6

In [7]:
# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links)

12x12 sparse matrix with 30 Int64 entries:
	[2 ,  1]  =  3
	[3 ,  1]  =  5
	[1 ,  2]  =  1
	[6 ,  2]  =  14
	[1 ,  3]  =  2
	[4 ,  3]  =  8
	[12,  3]  =  29
	[3 ,  4]  =  6
	[5 ,  4]  =  11
	[11,  4]  =  26
	⋮
	[9 ,  8]  =  22
	[5 ,  9]  =  13
	[8 ,  9]  =  20
	[10,  9]  =  24
	[9 , 10]  =  23
	[11, 10]  =  27
	[4 , 11]  =  10
	[10, 11]  =  25
	[12, 11]  =  30
	[3 , 12]  =  7
	[11, 12]  =  28

In [8]:
graph

Directed Graph (12 vertices, 30 edges)

In [9]:
function BPR(x)
    # travel_time = free_flow_time .* ( 1.0 + B .* (x./capacity).^power )
    # generalized_cost = travel_time + toll_factor *toll + distance_factor * link_length
    # return generalized_cost

    bpr = similar(x)
    for i=1:length(bpr)
        bpr[i] = free_flow_time[i] * ( 1.0 + B[i] * (x[i]/capacity[i])^power[i] )
        bpr[i] += toll_factor *toll[i] + distance_factor * link_length[i]
    end
    return bpr
end

BPR (generic function with 1 method)

In [10]:
function all_or_nothing(travel_time)
    state = []
    path = []
    x = zeros(size(start_node))

    for r=1:size(travel_demand)[1]
        # for each origin node r, find shortest paths to all destination nodes
        state = dijkstra_shortest_paths(graph, travel_time, r)

        for s=1:size(travel_demand)[2]
            # for each destination node s, find the shortest-path vector
            # load travel demand
            x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
        end
    end

    return x
end

all_or_nothing (generic function with 1 method)

In [11]:
# Finding a starting feasible solution
travel_time = BPR(zeros(number_of_links))
xl = all_or_nothing(travel_time)

30-element Array{Float64,1}:
  1900.0
  3500.0
  1900.0
  3900.0
  3500.0
  6100.0
  3600.0
  6100.0
 11300.0
  3800.0
 11300.0
  8700.0
  7900.0
     ⋮  
  6500.0
  4900.0
  7900.0
  4900.0
 13500.0
 13500.0
  8000.0
  3900.0
  7900.0
  4800.0
  3600.0
  4800.0

In [12]:
max_iter_no = 1e12
l = 1
#average_excess_cost = 1
tol = 1e-6

while l < max_iter_no
    l += 1
    
    xl_old = xl
    
    # Finding yl
    travel_time = BPR(xl)
    
    yl = all_or_nothing(travel_time)
    
    assert(yl != xl)
    
    xl = xl + (yl - xl)/l
    
    xl_new = xl
    
    relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

    if l % 10000 == 0
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
    end

    if relative_gap < tol 
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
        break
    end
    
end

l = 10000------------------------------------------------
relative_gap is 2.7587705621363506e-6
l = 20000------------------------------------------------
relative_gap is 2.6840722964007145e-6
l = 27578------------------------------------------------
relative_gap is 9.999674366381705e-7


In [13]:
outfile = open("./data_small_size/flows_converge_simplified.txt", "w")

write(outfile, join(("From", "to", "Volume_Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], xl[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)